# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and r_nc source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

First, the sample size is 2435, so it is considered large. Second, by the trial design, observations are independent of each other, so Central Limit Theorem applies. The goal of the study is to compare responses to resumes based on their perceived race, in particular to detect if there is race discrimination of the callbacks to submitted resumes. This can be cast into a problem of analysis of means of two groups, whites and blacks. So, two sample t-test is appropriate.

To determine if there is racial discrimination between two groups, we form a null hypothesis $H_o$, stating that there is no difference in the means between mean response for whites, $\mu_w$, and blacks, $\mu_b$. The alternative hypothesis $H_a$ is that there is a difference. Formally: 

$H_o: \mu_w = \mu_b$

$H_a: \mu_w \neq \mu_b$

In [1]:
import pandas as pd
import numpy as np
#from scipy import stats

import scipy.stats as sts
import statsmodels.stats.api as sms

C:\Users\HP\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
r_nc = pd.io.stata.read_stata('us_job_market_discrimination.dta')   #"r_nc"=resume-nr-column

# r_nc.to_csv('racial_disc.csv')   # , sep='\t' # save to csv file to examine data visually

In [3]:
# number of callbacks for black-sounding names
print(sum(r_nc[r_nc.race=='b'].call))
print(sum(r_nc[r_nc.race=='w'].call))

157.0
235.0


In [11]:
# create two separate dataframes for whites and blacks
rw_nc=r_nc.loc[r_nc['race'] == 'w']     # rw_nc.head()    # "rw_nc"data on resumes of whites
rw_n=rw_nc['call']                      # rw_n.head()  sum(rw_n)
rw_n=rw_n.reset_index(drop=True)        # rw_n = pd.DataFrame(rw_n.values);   #
rw1_n=rw_n   # *len(rw_n)   #/len(rw_n)               # get the population proportion

rb_nc=r_nc.loc[r_nc['race'] == 'b']    # rw_nc.head()   # "rb_nc"=data on resumes of blacks
rb_n=rb_nc['call']   # rw_n.head()  sum(rb_n)
rb_n=rb_n.reset_index(drop=True)
rb1_n=rb_n   # *len(rw_n)   #/len(rb_n)

# create a call series (for "call" column) of rw_n - rb_n   #s = pd.concat([df[i] for i in df], ignore_index = True)
dr_n=rw_n - rb_n    # rwb_n[:100]
dr1_n=dr_n   # *len(rw_n)   #/len(dr_n)

In [26]:
# get the callback frequency in % for whites and blacks    #rw1_n.mean()/len(rw1_n) #rw_n.mean()*len(rw_n)   # len(rw_n)
mw=round(rw_n.mean(),3); mb=round(rb_n.mean(),3);    # "mw"=mean-for-whites, "mb"=mean-for-blacks
print('Callback mean frequency for whites=',mw,', for blacks=',mb)

Callback frequency for whites= 0.097 , for blacks= 0.064


Let us determine the effect size of our study, which is the difference in means of callback frequency for white and black resumes, $\Delta \mu$.

In [32]:
# get the difference in means by taking the mean of W-B random variable series dr_n
dm=round(dr_n.mean(),3)                             ## *len(rw_n)    # rw_n.mean()*len(rw_n)
dm_err=round(sts.sem(dr_n),3)    #  compute standard error of the mean, should be the same as sqrt(var_w + var_b)   

print('The difference in means is ',dm,' which is ',dm/mb*100,'%',', with standard error SE=',dm_err*100,'%')

The difference in means is  0.032  which is  50.0 % , with standard error SE= 0.6 %


We see that the 50% difference between callback frequency for white and black resumes is substantial. So, we might want to reject the null hypothesis. Let us perform a two sample t-test to see, if it supports rejecting the null hypothethis.

In [43]:
# perform a two-sample t-test on white and black samples    #r_nc.head()  list(r_nc)               #rvs1 = stats.norm.rvs(loc=5,scale=10,size=500)  # rvs1[:10]
sts.ttest_ind(rw_n,rb_n)                                    # , equal_var = False

Ttest_indResult(statistic=4.1147052908617514, pvalue=3.9408021031288859e-05)

We see that the probability of obtaining result as extreme as 50% under null hypothesis is $p-value=3.9*10^{-5}$, very small. Therefore, we reject $H_o$. 
Next, let us determine the 95% confidence interval of our $\Delta \mu$ result.

In [42]:
# let us get the two sample t-test confidence interval
cm = sms.CompareMeans(sms.DescrStatsW(rw_n), sms.DescrStatsW(rb_n))   ## sts.t.interval(0.95, len(dr_n)-1, loc=np.mean(dr_n), scale=sts.sem(dr_n))
mu_l, mu_r=cm.tconfint_diff(usevar='unequal')
mu_l=round(mu_l,3); mu_r=round(mu_r,3);

print('95% Confidence interval is [',mu_l,',',mu_r,']')

95% Confidence interval is [ 0.017 , 0.047 ]


We compared callback frequencies for white and black sounding names on the job seeking resumes in Chicago and Boston areas. We found that there is a substantial difference of 50% between callback frequencies. This is a large effect. By performing further statistical analysis in the form of two sample t-test, we determined that this is a statistically significant effect with very small p-value of $3.9*10^{-5}$. This implies that the race/name is one of the important factors in the callback success, but it is not the only one. There are other factors which change the callback frequency substantially, such as years of experience, quality of the resume, mailing address etc. This suggests further analysis in the form of correlation analysis and logistic regression.  

                                                                          Author: Vlad Savchenko